In [5]:
#Journal of MIS 
#using Beautiful soup not getting good results
import requests
import csv
from bs4 import BeautifulSoup
url='https://www.jmis-web.org/'
response=requests.get(url)
with open('Journal_of_MIS.csv',mode='a',newline='',encoding='utf-8') as file:
    writer=csv.writer(file)
    writer.writerow(['title','url'])
if response.status_code==200:
    soup=BeautifulSoup(response.text,'html.parser')
    volume_titles=soup.select('div.margin-bottom-15 h4 > a')
    for title in volume_titles:
        article_url=title.get('href')
        article_text=title.text.strip()
        try:
            with open('Journal_of_MIS.csv',mode='a',newline='',encoding='utf-8') as file:
                writer=csv.writer(file)
                writer.writerow([article_text,article_url])
            print("done")
        except Exception as e:
            print(e)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")   

In [3]:
#this is for scrape the articles of volumnes and issues.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import csv
import time
#intialize the chrome webdriver
driver=webdriver.Chrome()
driver.get('https://www.tandfonline.com/toc/mmis20/41/4?nav=tocList')
# Wait for the page to fully load
driver.implicitly_wait(20)
def getVolumeIssues(url):
    driver=webdriver.Chrome()
    driver.implicitly_wait(20)
    try:
        if not url:
            return 0
        driver.get(url)
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        
        
        vol_title=driver.find_element(By.CSS_SELECTOR,'div.toc-title')
        title=vol_title.text.strip()
    
       
    
        articles=driver.find_elements(By.CSS_SELECTOR,'div.art_title')
        for article in articles:
            try:
               # Wait for the element to be present before trying to access it
                url_ele = WebDriverWait(article, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'a.ref'))
                )
                # Get the URL
                article_url = url_ele.get_attribute('href')
                article_title = article.find_element(By.CSS_SELECTOR,'span.hlFld-Title')
                title_text = article_title.text.strip()
            except Exception as e:
                print("error at article url",e)
                continue
            with open('Journal_of_MIS_new.csv', mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow([title, article_url, title_text])
                    file.flush() 
    except Exception as e:
        print("An error occurred while processing volume issues:", e)
    driver.quit()
   
    
def findIssues(url):
     #find the issues
    driver=webdriver.Chrome()
    driver.get(url)
    print(url)
    driver.implicitly_wait(20)
    try:
        issue_ele = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.loi-issues-scroller'))
            )
        issue_links = issue_ele.find_elements(By.CSS_SELECTOR, 'a')
        for issue in issue_links:
            href = issue.get_attribute('href')
            getVolumeIssues(href)
    except Exception as e:
        print(f"Error while finding issues for {url}: {e}")
        getVolumeIssues(url)
    driver.quit()
urls=['https://www.tandfonline.com/toc/mmis20/41/3?nav=tocList']
for i in urls:
    findIssues(i)
# years_ele=driver.find_element(By.CSS_SELECTOR,'div.yearSliderInner')
# years=years_ele.find_elements(By.CSS_SELECTOR,'a.expander')

# for year in years:
#     url=year.get_attribute('href')
#     findIssues(url)


driver.quit()

https://www.tandfonline.com/toc/mmis20/41/3?nav=tocList
error at article url Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6B6CC80D5+2992373]
	(No symbol) [0x00007FF6B695BFD0]
	(No symbol) [0x00007FF6B67F590A]
	(No symbol) [0x00007FF6B684926E]
	(No symbol) [0x00007FF6B684955C]
	(No symbol) [0x00007FF6B683C6CC]
	(No symbol) [0x00007FF6B686F3AF]
	(No symbol) [0x00007FF6B683C596]
	(No symbol) [0x00007FF6B686F580]
	(No symbol) [0x00007FF6B688F584]
	(No symbol) [0x00007FF6B686F113]
	(No symbol) [0x00007FF6B683A918]
	(No symbol) [0x00007FF6B683BA81]
	GetHandleVerifier [0x00007FF6B6D26A2D+3379789]
	GetHandleVerifier [0x00007FF6B6D3C32D+3468109]
	GetHandleVerifier [0x00007FF6B6D30043+3418211]
	GetHandleVerifier [0x00007FF6B6ABC78B+847787]
	(No symbol) [0x00007FF6B696757F]
	(No symbol) [0x00007FF6B6962FC4]
	(No symbol) [0x00007FF6B696315D]
	(No symbol) [0x00007FF6B6952979]
	BaseThreadInitThunk [0x00007FFFFF687344+20]
	RtlUserThreadStart [0x00007FFFFF7C26B1+33]

error at article url Message

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

import csv
import time
euro_data=pd.read_excel('Journal_of_MIS1.xlsx')
def getAuthorsData(authors):
    auth_data=[]
    name,url,desc,email='','','',''
    for auth in authors:
        hover = ActionChains(driver).move_to_element(auth)
        hover.perform()
        try:
            auth_details=auth.find_element(By.CSS_SELECTOR,'a.author')
            name=auth_details.text.strip()
           
            url=auth_details.get_attribute('href')
        except Exception as e:
            print('error at name and email of author',e)
        try:
            # Wait for the overlay description to appear
            desc = auth.find_element(By.CSS_SELECTOR,'span.overlay').text.strip()
        except Exception as e:
            print("error at description",e)
        try:
            email = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'span.corr-email a'))
            ).text.strip()
        except Exception as e:
            print("error at author email",e)
        auth_data.append([name,url,desc,email])
        ActionChains(driver).move_to_element(driver.find_element(By.TAG_NAME, 'body')).perform()
    return auth_data
        
with open('Journal_of_MIS_articles_data_new.csv',mode='a',newline='',encoding='utf-8') as file:
    writer=csv.writer(file)
    writer.writerow(['Journal_Title',"URL", "Article_Title",'Volume_Issue','Month_Year','Abstract','Keywords','Author_name','Author_Address','Author_email'])  # Write the header row
for index,row in euro_data.iterrows():
    final_data=[]
    driver=webdriver.Chrome()
    driver.get(row['URL'])
    driver.implicitly_wait(10)
    try:
        #to avoid no such element error use webdriverwait
        vol_year_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.issue-heading'))
        )
        vol_year = vol_year_element.text.strip()
        
        vol_issue_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.issue-heading a'))
        )
        vol_issue = vol_issue_element.text.strip()
    except Exception as e:
        print("Error occured",e)
    try:
        abstract=driver.find_element(By.CSS_SELECTOR,'div.hlFld-Abstract p.last').text.strip()
    except Exception as e:
        abstract='NA'
        print("error at abstraction",e)
    try:
        keywordlist=[]
        keyword_list_element=driver.find_element(By.CSS_SELECTOR,'div.hlFld-KeywordText')
        keyword_list=keyword_list_element.find_elements(By.CSS_SELECTOR,'a.kwd-btn')
        for key in keyword_list:
            keyword=key.text.strip()
            keywordlist.append(keyword)
    except Exception as e:
        print(e)
    final_data=[row['Title'],row['URL'],row['Article_Title'],vol_issue,vol_year,abstract,keywordlist]
    try:
        authors=driver.find_elements(By.CSS_SELECTOR,'div.addAuthorInfo')
        # authors=driver.find_elements(By.CSS_SELECTOR,'div.entryAuthor')
        # authdata=getAuthorsData(authors)
        for auth in authors:
            try:
                auth_title=auth.find_element(By.CSS_SELECTOR,'h4').text.strip()
            except Exception as e:
                auth_title='NA'
            try:
                auth_email=auth.find_element(By.CSS_SELECTOR,'div.NLM_bio a').text.strip()
            except Exception as e:
                auth_email='NA'
            try:
                auth_desc=auth.find_element(By.CSS_SELECTOR,'div.NLM_bio p').text.strip()
            except Exception as e:
                auth_desc='NA'
            with open('Journal_of_MIS_articles_data_new.csv', mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(final_data + [auth_title,auth_desc,auth_email])
                file.flush()
    except Exception as e:
            print(f"Error processing author data on {row['URL']}: {e}")
   
    driver.quit()


    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    


error at abstraction Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.hlFld-Abstract p.last"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6603F80D5+2992373]
	(No symbol) [0x00007FF66008BFD0]
	(No symbol) [0x00007FF65FF2590A]
	(No symbol) [0x00007FF65FF7926E]
	(No symbol) [0x00007FF65FF7955C]
	(No symbol) [0x00007FF65FFC27D7]
	(No symbol) [0x00007FF65FF9F3AF]
	(No symbol) [0x00007FF65FFBF584]
	(No symbol) [0x00007FF65FF9F113]
	(No symbol) [0x00007FF65FF6A918]
	(No symbol) [0x00007FF65FF6BA81]
	GetHandleVerifier [0x00007FF660456A2D+3379789]
	GetHandleVerifier [0x00007FF66046C32D+3468109]
	GetHandleVerifier [0x00007FF660460043+3418211]
	GetHandleVerifier [0x00007FF6601EC78B+847787]
	(No symbol) [0x00007FF66009757F]
	(No symbol) [0x00007FF660092FC4]
	(No symbol) [0x0